In [1]:
import db.knowhere_db as kdb
import pipeline.pipeline as pipeline
import pandas as pd
import numpy as np
import pickle
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
reader = kdb.Reader('knowhere')

In [3]:
glen_H_data_raw = reader.get_dataframe_pivoted(collection='iphone_test3', username='glen', commute=True,\
                                               min_date='2017-3-20', max_date='2017-03-24')

In [4]:
glen_H_data = glen_H_data_raw.dropna(subset=['Acceleration z', 'Acceleration x', 'Acceleration y'], how='all')


In [13]:
glen_commute_labels = pickle.load( open("glen_commute_labels.p", "rb" ))

In [14]:
def add_acceleration_magnitude(df):
    "Adds Acceleration magnitude to a dataframe. First 3 columns must be acceleration vectors. Must be 3D."
    from math import sqrt

    accel_mag = []
    
    for row in range(len(df)):
        accel_mag.append(sqrt(df.iloc[row, 0]**2 + df.iloc[row, 1]**2 + df.iloc[row, 2]**2))

    df['accel_mag'] = accel_mag
    return 'finished!'

def add_classifications(df, min_date, max_date, events):    
    '''Pass events as a list of tuples (datetime, eventcode) to classify different events'''
    for days_events in events.iterkeys():
        if pd.to_datetime(days_events) in pd.date_range(start=min_date, end=max_date, freq='D'):
            for event in events[days_events]:
                df.loc[days_events + " " + event[0], 'classification'] = event[1]
    df = df.fillna(method='ffill', inplace = True)
    #df['classification'] = df['classification'].fillna(method='ffill', inplace = True)
    return "Finished!"

def label_raw_iphone(df, min_date, max_date, events, username, accel_mag = False):
    df.index = pd.to_datetime(df.index)
    if accel_mag == True:
        add_acceleration_magnitude(df)
    df['classification'] = None
    add_classifications(df, min_date, max_date, events)
    df.to_csv("Labeled_" + username + "_" + min_date + " to " + max_date + ".csv")
    return df

In [15]:
label_raw_iphone(glen_H_data, "2017-03-19", "2017-03-23", glen_commute_labels , username = 'glen')

/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/andrew/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

,GPS Horizontal Accuracy,GPS Longitude,GPS Vertical Accuracy,Magnetometer z,Altimeter (Barometer) Pressure,GPS Latitude,GPS Altitude,Magnetometer y,Magnetometer x,Altimeter (Barometer) Relative Altitude,...,Gyrometer y,Gyrometer z,Gyrometer x,Gravity z,Gravity y,Gravity x,Acceleration z,Acceleration x,Acceleration y,classification
2017-03-20 07:33:33,65.0,-73.753620,10.0,-448.443573,101.280930,40.736642,49.617031,138.553955,-53.052246,0.000000,...,-0.212767,0.489268,-1.204461,0.569031,-0.819343,0.069859,-1.372254,-0.152821,0.563072,D
2017-03-20 07:33:37,50.0,-73.753751,8.0,-448.443573,101.285233,40.736369,45.059113,138.553955,-53.052246,-0.358571,...,-0.212767,0.489268,-1.204461,-0.015207,-0.970466,-0.240759,-0.714468,0.377050,0.415137,D
2017-03-20 07:33:42,10.0,-73.753777,3.0,-448.443573,101.288124,40.736884,48.498138,138.553955,-53.052246,-0.598953,...,-0.212767,0.489268,-1.204461,-0.952815,-0.302552,-0.024629,-0.189779,0.012040,0.095826,D
2017-03-20 07:33:46,10.0,-73.753710,3.0,-448.443573,101.287880,40.736812,48.162506,138.553955,-53.052246,-0.578837,...,-0.212767,0.489268,-1.204461,-0.991672,-0.122558,0.039572,0.194141,0.133631,-0.102296,D
2017-03-20 07:33:51,10.0,-73.753704,4.0,-448.443573,101.284981,40.736812,49.439728,138.553955,-53.052246,-0.337450,...,-0.212767,0.489268,-1.204461,-0.953794,-0.053384,-0.295682,0.066937,-0.046038,0.116571,D
2017-03-20 07:33:55,10.0,-73.753737,4.0,-446.808716,101.281601,40.736793,48.868744,134.090012,-89.450562,-0.055823,...,-0.212767,0.489268,-1.204461,-0.964337,0.045806,0.260683,-0.182697,-0.107973,-0.140334,D
2017-03-20 07:34:00,10.0,-73.753733,4.0,-442.558044,101.283562,40.736761,49.204132,139.584091,-48.244934,-0.219268,...,-0.212767,0.489268,-1.204461,-0.231169,0.164317,0.958937,-0.009279,-0.045027,0.000996,D
2017-03-20 07:34:04,10.0,-73.753722,4.0,-442.558044,101.285690,40.736722,49.528961,139.584091,-48.244934,-0.396288,...,-0.212767,0.489268,-1.204461,-0.493534,-0.426474,0.757987,-0.222485,0.091546,-0.424249,D
2017-03-20 07:34:09,10.0,-73.753690,4.0,-442.558044,101.286499,40.736679,49.136444,139.584091,-48.244934,-0.463676,...,-0.212767,0.489268,-1.204461,-0.866083,-0.483445,0.127207,-0.218771,-0.001429,0.000199,D
2017-03-20 07:34:13,10.0,-73.753681,4.0,-442.558044,101.297173,40.736662,49.671539,139.584091,-48.244934,-1.352247,...,-0.747892,0.672498,-0.189776,-0.858767,-0.106828,-0.501106,-0.174451,0.035286,-0.032134,D


In [16]:
glen_H_data = glen_H_data[['Acceleration x','Acceleration y','Acceleration z', 'Altimeter (Barometer) Pressure',\
                               'Microphone Left Channel Level', 'Microphone Right Channel Level','Magnetometer x',\
                               'Magnetometer y','Magnetometer z','Gyrometer x','Gyrometer y',\
                               'Gyrometer z', 'classification']]
glen_H_data = glen_H_data.dropna()
glen_H_data.iloc[:,0:11].astype(float)


,Acceleration x,Acceleration y,Acceleration z,Altimeter (Barometer) Pressure,Microphone Left Channel Level,Microphone Right Channel Level,Magnetometer x,Magnetometer y,Magnetometer z,Gyrometer x,Gyrometer y
2017-03-20 07:34:13,0.035286,-0.032134,-0.174451,101.297173,-22.18490,-22.18490,-48.244934,139.584091,-442.558044,-0.189776,-0.747892
2017-03-20 07:34:18,0.045886,-0.052843,0.180941,101.301842,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667
2017-03-20 07:34:22,0.037488,-0.080072,0.024071,101.297859,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667
2017-03-20 07:34:27,0.104773,-0.102392,0.040786,101.294731,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667
2017-03-20 07:34:31,0.196326,-0.233689,-0.079535,101.297264,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667
2017-03-20 07:34:36,-0.050452,-0.124135,-0.159513,101.299042,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667
2017-03-20 07:34:40,-0.071883,-0.096012,0.228122,101.298973,-5.64593,-5.64593,-60.778305,147.310150,-456.454407,0.186489,1.126667
2017-03-20 07:34:45,-0.188200,0.358660,-1.074619,101.302010,-5.64593,-5.64593,-55.284225,109.881699,-443.048523,0.186489,1.126667
2017-03-20 07:34:49,-0.124106,0.023692,-0.032372,101.300888,-5.64593,-5.64593,-55.284225,109.881699,-443.048523,0.186489,1.126667
2017-03-20 07:34:54,-0.108972,-0.171239,0.021897,101.301849,-5.64593,-5.64593,-55.284225,109.881699,-443.048523,0.186489,1.126667


In [17]:
glen_H_data.groupby("classification").size()

classification
D      1749
E_D     405
E_U     480
S       251
S_U      31
T_D    1159
T_E       2
T_S     747
T_T      40
W      1328
dtype: int64

In [18]:
glen_H_data['Acceleration'] =  np.sqrt(glen_H_data['Acceleration x']**2 + glen_H_data['Acceleration y']**2 +\
                                       glen_H_data['Acceleration z']**2)
glen_H_data['Magnetometer'] =  np.sqrt(glen_H_data['Magnetometer x']**2 + glen_H_data['Magnetometer y']**2 +\
                                       glen_H_data['Magnetometer z']**2)
glen_H_data['Gyrometer']    =  np.sqrt(glen_H_data['Gyrometer x']**2 + glen_H_data['Gyrometer y']**2 +\
                                       glen_H_data['Gyrometer z']**2)
glen_H_data['Microphone']    =  (glen_H_data['Microphone Left Channel Level'] +\
                                glen_H_data['Microphone Right Channel Level'])/2
glen_H_data = glen_H_data[['Acceleration','Magnetometer','Gyrometer','Microphone','Altimeter (Barometer) Pressure', 'classification']]

In [19]:
# set window
window = 5
# Rolling Means
glen_H_data['RollingMeanAcceleration'] = pd.rolling_mean(glen_H_data['Acceleration'], window)
glen_H_data['RollingMeanMagnetometer'] = pd.rolling_mean(glen_H_data['Magnetometer'], window)
glen_H_data['RollingMeanGyrometer'] = pd.rolling_mean(glen_H_data['Gyrometer'], window)
glen_H_data['RollingMeanMicrophone'] = pd.rolling_mean(glen_H_data['Microphone'], window)
glen_H_data['RollingMeanAltimeter'] = pd.rolling_mean(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling 75th percentile
glen_H_data['Rolling75thAcceleration'] = pd.rolling_quantile(glen_H_data['Acceleration'], window, 0.75)
glen_H_data['Rolling75thMagnetometer'] = pd.rolling_quantile(glen_H_data['Magnetometer'], window, 0.75)
glen_H_data['Rolling75thGyrometer'] = pd.rolling_quantile(glen_H_data['Gyrometer'], window, 0.75)
glen_H_data['Rolling75thMicrophone'] = pd.rolling_quantile(glen_H_data['Microphone'], window, 0.75)
glen_H_data['Rolling75thAltimeter'] = pd.rolling_quantile(glen_H_data['Altimeter (Barometer) Pressure'], window, 0.75)
# Rolling Max
glen_H_data['RollingMaxAcceleration'] = pd.rolling_max(glen_H_data['Acceleration'], window)
glen_H_data['RollingMaxMagnetometer'] = pd.rolling_max(glen_H_data['Magnetometer'], window)
glen_H_data['RollingMaxGyrometer'] = pd.rolling_max(glen_H_data['Gyrometer'], window)
glen_H_data['RollingMaxMicrophone'] = pd.rolling_max(glen_H_data['Microphone'], window)
glen_H_data['RollingMaxAltimeter'] = pd.rolling_max(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling Min
glen_H_data['RollingMinAcceleration'] = pd.rolling_min(glen_H_data['Acceleration'], window)
glen_H_data['RollingMinMagnetometer'] = pd.rolling_min(glen_H_data['Magnetometer'], window)
glen_H_data['RollingMinGyrometer'] = pd.rolling_min(glen_H_data['Gyrometer'], window)
glen_H_data['RollingMinMicrophone'] = pd.rolling_min(glen_H_data['Microphone'], window)
glen_H_data['RollingMinAltimeter'] = pd.rolling_min(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling st dev
glen_H_data['RollingSDAcceleration'] = pd.rolling_std(glen_H_data['Acceleration'], window)
glen_H_data['RollingSDMagnetometer'] = pd.rolling_std(glen_H_data['Magnetometer'], window)
glen_H_data['RollingSDGyrometer'] = pd.rolling_std(glen_H_data['Gyrometer'], window)
glen_H_data['RollingSDMicrophone'] = pd.rolling_std(glen_H_data['Microphone'], window)
glen_H_data['RollingSDAltimeter'] = pd.rolling_std(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling var
glen_H_data['RollingVarAcceleration'] = pd.rolling_var(glen_H_data['Acceleration'], window)
glen_H_data['RollingVarMagnetometer'] = pd.rolling_var(glen_H_data['Magnetometer'], window)
glen_H_data['RollingVarGyrometer'] = pd.rolling_var(glen_H_data['Gyrometer'], window)
glen_H_data['RollingVarMicrophone'] = pd.rolling_var(glen_H_data['Microphone'], window)
glen_H_data['RollingVarAltimeter'] = pd.rolling_var(glen_H_data['Altimeter (Barometer) Pressure'], window)

/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/home/andrew/anaconda2/lib/python2.7/site-packages/ipykernel/__main_

In [20]:
glen_H_data = glen_H_data.dropna()
glen_H_data_re = glen_H_data.iloc[:,range(0,5) + range(6,36) + [5]]

In [21]:
# load dataset
dataset = glen_H_data_re.values
X = dataset[:,0:34]
Y = dataset[:,35]
X, Y = shuffle(X, Y, random_state=3719392992)
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# One Hot Encode
#lb = LabelBinarizer()
#lb.fit(encoded_Y)
#dummy_y = lb.fit_transform(encoded_Y)

In [23]:
# build a classifier
RF_Class = RandomForestClassifier(n_estimators=200)
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [2, 3, 2, 10, 13, 27, None],
              "max_features": sp_randint(1,35),
              "min_samples_split": sp_randint(2, 100),
              "min_samples_leaf": sp_randint(1, 100),
              "bootstrap": [True],
              "criterion": ["gini"]}

# run randomized search
n_iter_search = 5
random_search = RandomizedSearchCV(RF_Class, param_distributions=param_dist, n_iter=n_iter_search)

start = time()
random_search.fit(X, Y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 95.71 seconds for 5 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.862 (std: 0.011)
Parameters: {'bootstrap': True, 'min_samples_leaf': 1, 'min_samples_split': 77, 'criterion': 'gini', 'max_features': 26, 'max_depth': None}

Model with rank: 2
Mean validation score: 0.830 (std: 0.005)
Parameters: {'bootstrap': True, 'min_samples_leaf': 29, 'min_samples_split': 67, 'criterion': 'gini', 'max_features': 8, 'max_depth': 10}

Model with rank: 3
Mean validation score: 0.785 (std: 0.021)
Parameters: {'bootstrap': True, 'min_samples_leaf': 51, 'min_samples_split': 69, 'criterion': 'gini', 'max_features': 24, 'max_depth': 27}



In [24]:
# CV the Random Firest Model with the best paramerters
Rand_Forest = RandomForestClassifier(n_estimators=1000, bootstrap=True,min_samples_leaf=83,min_samples_split=11,\
                                     criterion='gini', max_features=18,max_depth=27)
scores = cross_val_score(Rand_Forest, X, Y, cv=2)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.71 (+/- 0.04)


In [1]:
GBM = GradientBoostingClassifier(n_estimators=1000)
scores = cross_val_score(GBM, X, Y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

NameError: name 'GradientBoostingClassifier' is not defined

In [ ]:
KNN = KNeighborsClassifier(n_neighbors=3)
GNB = GaussianNB()
lr = LogisticRegression()
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
stacked_class = StackingCVClassifier(classifiers=[GBM, Rand_Forest], 
#                                     use_probas=True,
                                     meta_classifier=lr)
print('5-fold cross validation:\n')

for clf, label in zip([GBM, Rand_Forest, stacked_class], 
                      ['GBM', 
                       'Rand_Forest',
                       'StackingClassifier']):


    scores = model_selection.cross_val_score(clf, X, encoded_Y, cv=5, scoring='neg_log_loss')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))


In [ ]:
x_train,x_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.33, random_state=41010392)
lr.fit(x_train, y_train)
Y_pred = lr.predict(x_test)
print np.unique(Labels)
confusion_matrix(y_test,Y_pred)

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.33, random_state=41010392)
NN.fit(x_train, y_train)
Y_pred = NN.predict(x_test)
print np.unique(Labels)
confusion_matrix(y_test,Y_pred)

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.33, random_state=54535353)
Rand_Forest.fit(x_train, y_train)
Y_pred = Rand_Forest.predict(x_test)
print np.unique(Labels)
confusion_matrix(y_test,Y_pred)

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.33, random_state=41010392)
GBM.fit(x_train, y_train)
Y_pred = GBM.predict(x_test)
print np.unique(Labels)
confusion_matrix(y_test,Y_pred)

In [ ]:
Labels =encoder.inverse_transform(encoded_Y)
print np.unique(Labels)
confusion_matrix(encoded_Y,Y_pred)